# Computing transport coefficients with Molly

CECAM Summer School: "Sampling High-Dimensional Probability Measures with Applications in (Non)Equilibrium Molecular Dynamics and Statistics" - 07/07 to 07/11 2025

**Author: Noé Blassel**

The aim of this practical session is to compute some transport coefficients with [Molly.jl](https://github.com/JuliaMolSim/Molly.jl).

We focus on the shear viscosity of liquid argon, using the transverse force field method analyzed in ([Joubaud & Stoltz, 2012 (JS12)](https://epubs.siam.org/doi/abs/10.1137/110836237)).

The underlying **equilibrium** dynamics is the underdamped Langevin dynamics with damping parameter $\gamma$ (with dimensions $\mathbf{M\cdot T}^{-1}$).

$$
\tag{3}
\begin{align*}
    \begin{cases}
        dq_t = M^{-1}p_t \, dt, \\
        dp_t = -\nabla V(q_t) \, dt - \gamma M^{-1}p_t \, dt+\sqrt{\dfrac{2\gamma}{\beta}} dW_t.
    \end{cases}
\end{align*}
$$

## Theory

The STF method proceeds by analogy with Newton's macroscopic law of viscosity, for a fluid subjected to a shear force $f$ directed along the longitudinal $x$-coordinate, but varies in intensity in the transverse $y$-coordinate. At the continuum level, the shear viscosity~$\mu$ is defined via the constitutive relation
$$
    \sigma_{xy} = -\mu\frac{\mathrm d u_x}{\mathrm d y},
$$
where $\sigma_{xy}$ is the $(x,y)$ component of the local stress tensor, and $u_x$ is the local $x$-velocity field of the fluid. Both $\sigma_{xy}$ and $u_x$ are functions of the $y$ position in the fluid.

Microscopically, the shearing effect is created by a non-conservative force field:
$$
    \forall\,1\leq j\leq N,\quad F(q)_{j,x} = f(q_{j,y}),\quad F(q)_{j,y}=F(q)_{j,z}=0.
$$

This forcing field acts on the $x$-momentum of each particle, depending on the corresponding $y$-position, according to a fixed forcing profile $f(y)$. The STF method derives its name from the standard choice $f(y) = \sin(2\pi y/L)$ (which we use in this notebook), although other profiles can be considered. Here is a movie of a **very** forced system, viewed from above the $(x,y)$-plane.

<p align="center">
    <img src="sin_visc3.gif" controls title="Title"></img>
</p>

The microscopic realization of the Newton relation relies on definitions of the velocity and shear-stress profiles $u_x$, $\sigma_{xy}$. Rigorously, these are defined as linear responses (for limits) of $y$-dependent observables, see [JS12](https://epubs.siam.org/doi/abs/10.1137/110836237) for a derivation inspired by the Irving--Kirkwood method.

Using linear response theory, one can show that the velocity and shear-stress linear response profiles $u_x,\sigma_{xy}$ and forcing profile $f$ are related exactly via
$$
    \bar{\rho}\frac{\mathrm{d}\sigma_{xy}(y)}{\mathrm{d} y}  + \gamma u_x(y) = f(y),
$$
where $\bar{\rho}= N/L^3$ is the **particle** density.

Formally substituting in Newton's relation, we then get
$$
    -\bar{\rho}^{-1}\mu\frac{\mathrm{d}^2 u_x(y)}{\mathrm{d} y^2} + \gamma u_x(y) = f(y),
$$
and taking Fourier coefficients with respect to $y$ on the periodic domain $L\mathbb T_y$, the viscosity $\mu$ is then given by
$$
    \mu = \frac1{\bar{\rho}}\left(\frac{L}{2\pi}\right)^2\left(\frac{F_1}{U_1}-\gamma\right),
$$
where $F_1$, $U_1$ are the Fourier coefficients:
$$
F_1 = \frac1{L}\int_0^L f(y)\mathrm{e}^{2i\pi y/L}\,\mathrm{d} y,\qquad U_1 = \frac1{L}\int_0^L u_x(y)\mathrm{e}^{2i\pi y /L}\,\mathrm{d} y
$$

The unknown quantity is the Fourier coefficient $U_1$. One can show that $U_1$ is itself the linear response of a given observable, namely the ''empirical Fourier'' flux
$$
    R(q,p) = \frac1{Nm}\sum_{j=1}^N p_{j,x}\exp\left(\frac{2\mathrm{i}\pi q_{j,y}}{L}\right).
$$
We will determine $U_1$ using NEMD trajectory averages and the Green--Kubo formula with the conjugate flux:
$$
    S(q,p) = \frac{\beta}{m} F(q)^\top p. 
$$

We will use the features of Molly to implement both these methods.

## Preliminary steps

We start by importing the necessary packages, and define a function that will be used to pool simulation results from the participants. This cell should not be modified.

<details>
<summary> <b> Note </b> </summary>

If one are several packages are missing, you can install them by adding

`import Pkg; Pkg.add("<PackageName>")` in the cell below, or typing
`] add <PackageName>` in the Julia REPL.
</details>

In [ ]:
using Molly # for MD
using Statistics
using Plots,LaTeXStrings # to plot results
import Molly.AtomsCalculators # to define a general interaction
using Unitful # to handle units
using HTTP,JSON # for posting results
using ProgressMeter # for progress bars
using Measurements # to propagate uncertainties

## To communicate simulation results -- DO NOT MODIFY

"""
method::String, "GK" or "NEMD"
η::Float64 (0.0 if method="GK", or the magnitude of the forcing if method="NEMD")
steps::Int64 the number of simulation steps
R the estimation of the Fourier flux average response (a complex velocity)
ρ the system density
T the temperature
γ the friction parameter
N::Int64 the number of particles
"""
function post_result(method,η,steps,R,ρ,T,γ,N,U1)

    url = "https://api.sheetbest.com/sheets/ceb77507-1382-44e1-a3f0-39a0a8fbcebf"
    id = ENV["USERNAME"]

    # checks
    @assert (method == "NEMD" && isa(η,Float64)) || (method == "GK" && η == 0.0) "Invalid value for $η for specified method: $method"

    if method == "GK"
        @assert dimension(U1) == dimension(u"m/s") "Invalid dimension for U1: should be a velocity"
    elseif method == "NEMD"
        @assert dimension(R) == dimension(u"m/s") "Invalid dimension for R: should be a velocity"
    end

    @assert dimension(ρ) == dimension(u"kg/m^3") "Invalid dimension for ρ: should be a density"
    @assert dimension(T) == dimension(u"K") "Invalid dimension for T: should be temperature"
    @assert dimension(γ) == dimension(u"g/s") "Invalid dimension for γ: should be a mass per time"

    units = [u"Å/ps",u"Å/ps",u"g/cm^3",u"K",u"u/ps"]
    names = ["R","U1","rho","T","gamma"]
    params = Dict{String,Any}()

    for (unit,name,quantity)=zip(units,names,[R,U1,ρ,T,γ]) # ρ,T,γ get promoted to complex types here
        tmp = uconvert(unit,quantity) |> ustrip
        params[name] = isreal(tmp) ? real(tmp) : imag(tmp)
    end

    params["id"]= id
    params["method"] = method
    params["eta"] = η
    params["N"]= N
    params["steps"]= steps

    HTTP.post(url,["Content-type"=>"application/json"],body=JSON.json(params))

end

"""
Parses collected simulation results for a given thermodynamic condition.
"""
function fetch_results(ρ,T,γ,N)
    url = "https://api.sheetbest.com/sheets/ceb77507-1382-44e1-a3f0-39a0a8fbcebf"

    @assert dimension(ρ) == dimension(u"kg/m^3") "Invalid dimension for ρ: should be a density"
    @assert dimension(T) == dimension(u"K") "Invalid dimension for T: should be temperature"
    @assert dimension(γ) == dimension(u"g/s") "Invalid dimension for γ: should be a mass per time"

    units = [u"g/cm^3",u"K",u"u/ps",NoUnits]
    names = ["rho","T","gamma","N"]

    vals = ustrip.(units,[ρ,T,γ,N])

    raw = HTTP.get(url).body |> String |> JSON.parse

    gk_results = Vector{Float64}[]
    nemd_results = Vector{Float64}[]

    for d=raw
        if all(parse(Float64,d[n]) ≈ v for (n,v)=zip(names,vals))
            nsteps = parse(Int64,d["steps"])

            if d["method"] == "GK"
                imU1 = parse(Float64,d["U1"])
                push!(gk_results,[nsteps,imU1])

            elseif d["method"] == "NEMD"
                imR = parse(Float64,d["R"])
                η = parse(Float64,d["eta"])
                push!(nemd_results,[nsteps,η,imR])
            end
        end
    end

    return nemd_results,gk_results

end

---
## Method 1: NEMD simulation

For the nonequilibrium approach, we consider the case where the Langevin dynamics is perturbed by the <span style="color:blue">nongradient force</span> $\eta F$, with magnitude $\eta$:

$$
\begin{align*}
    \begin{cases}
        dq_t = M^{-1}p_t \, dt, \\
        dp_t = -\nabla V(q_t) \, dt \textcolor{blue}{+ \eta F(q_t) \, dt} - \gamma M^{-1}p_t \, dt + \sqrt{\dfrac{2\gamma}{\beta}} dW_t.
    \end{cases}
\end{align*}
$$
Here $\eta$ is a dimensionless quantity, and $F$ has the dimensions of a force.


**Linear response theory:** The perturbation $\eta F$ is expected to induce a response from the system, expressed as a steady-state average $\mathbb{E}_\eta(R)$ for the Fourier flux $R$. For small values of $\eta$, the response is linear in $\eta$. The *linear response* $U_1$ is thus defined as the proportionality constant between the perturbation and the response:

$$
    U_1 = \lim_{\eta\to 0} \frac{\mathbb{E}_\eta(R)}{\eta}.
$$
Check $U_1$ has the dimensions of a velocity.

For convenience, we also define a method `place_atoms_on_cubic_lattice` which initializes atomic coordinates at the cell centers of a simple orthogonal lattice. 

In [ ]:
"""
Place Nx*Ny*Nz atoms at the center of orthogonal cells
"""
function place_atoms_on_cubic_lattice(Nx::Integer, Ny::Integer, Nz::Integer, boundary)
    (Lx, Ly, Lz) = boundary.side_lengths
    dx, dy, dz = Lx / Nx, Ly / Ny, Lz / Nz
    return vec([SVector((i + 0.5) * dx - Lx / 2, (j + 0.5) * dy - Ly / 2, (k + 0.5) * dz - Lz / 2) for i = 0:Nx-1, j = 0:Ny-1, k = 0:Nz-1])
end

### 1.1 - Physical parameters
We define thermodynamic conditions. 

Define the length `L` of the periodic domain, in Angstrom (use [`Unitful`](https://painterqubits.github.io/Unitful.jl/stable/)) Define a [`CubicBoundary`](https://juliamolsim.github.io/Molly.jl/stable/api/#Molly.CubicBoundary) and initialize atomic coordinates using `place_atoms_on_cubic_lattice`.

<details>

<summary> <b> Answer </b> </summary>

```julia
const L = uconvert(u"Å", cbrt((N * m) / ρ))
boundary = CubicBoundary(L, L, L)
coords = place_atoms_on_cubic_lattice(Nps, Nps, Nps, boundary)
```

</details>

For convenience, we also define force and energy units.

In [ ]:
const kB = Unitful.k

const m = 39.95u"u"; # mass of an Argon atom.

const Nps = 6;
const N = Nps^3; # number of atoms

const T = 85u"K"; # temperature
const ρ = 1.41u"g/cm^3"; # density

const L = # fill me in!
boundary = # fill me in!
coords = # fill me in!

const force_unit = u"pN"
const energy_unit = u"pN*Å"

println("Argon system of N=$N atoms, in a periodic box of side-length L=$L.")


For liquid Argon, we use the classical Lennard-Jones interaction potential. The pairwise radial energy is given by:

$$
    v(r) = 4\epsilon\left[\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6\right],
$$

where
- $\sigma$ = distance at which $V(\sigma) = 0$;
- $\epsilon$ = depth of potential well.

<p align="center">
    <img src="LJ_plot2.png" alt="drawing" width="600"/>
</p>

We apply a cutoff to have a bounded-range interaction, allowing to speedup the force computation using a neighbor list.
The radial potential $V_\text{tr}(r)$ is $0$ beyond at cutoff radius $r_c>0$, with a piecewise linear term to ensure a continuous force:

$$
v_\text{tr}(r) = \chi_{r<r_c}\left(v(r) - (r-r_c)v'(r_c)\right),\qquad V(q)=\sum_{\substack{1\leq i< j\leq N\\|q_i-q_j|<r_c}} v_{\text{tr}}(|q_i-q_j|).
$$

### 1.2 - Equilibrium potential

- Define the [`LennardJones`](https://juliamolsim.github.io/Molly.jl/stable/api/#Molly.LennardJones) potential with cutoff [`ShiftedForce Cutoff`](https://juliamolsim.github.io/Molly.jl/stable/api/#Molly.ShiftedForceCutoff). Enable neighbor finding.

- Create a vector of `N` [`Atom`](https://juliamolsim.github.io/Molly.jl/stable/api/#Molly.Atom)s with the provided Lennard-Jones parameters.
- Define a [`CellListMapNeighborFinder`](https://juliamolsim.github.io/Molly.jl/stable/api/#Molly.CellListMapNeighborFinder) with the specified cutoff radius.

<details>
<summary>
<b> Answer </b>
</summary>

```julia
    atoms = [Atom(index=i, ϵ=ϵ, σ=σ, mass=m) for i = 1:N]
    cutoff = ShiftedForceCutoff(rc)
    lj_w_cutoff = LennardJones(cutoff=cutoff,use_neighbors=true)
    nf = CellListMapNeighborFinder(eligible=trues(N, N), unit_cell=boundary, n_steps=1, dist_cutoff=rc)
```
</details>

In [ ]:
# Lennard-Jones parameters
const ϵ = uconvert(energy_unit,0.0103u"eV");
const σ = 3.405u"Å";
const rc = 2.0 * σ;

atoms = # fill me in!
cutoff = # fill me in!
lj_w_cutoff = # fill me in!
nf = # fill me in!

### 1.3 - Nonequilibrium forcing

We now implement the extra forcing. We define a type, `NEMD_longitudinal_forcing`, with two attributes:
- `force_profile`, a function, which takes a length and returns a force.
- `η`, a dimensionless forcing magnitude, of floating-point type.
- For convenience, we also define energy units, `eV`, and force units, `eV/Å`.

Define a `NEMD_longitudinal_forcing` for the profile
$$
    f_y(u) = \sin\left(\frac{2\pi u}{L}\right),
$$

where `L` is the sidelength of the domain.

<details>

<summary><b>Answer</b></summary>

```julia

function get_forcing()
   η = ηmin + rand()*(ηmax-ηmin) # pick a random η

   println("Creating NEMD forcing with magnitude: $η")
   
   forcing = NEMD_longitudinal_forcing(y-> sin(2π * y / L)*force_unit, η)
    
    return forcing
end

```
</details>


In [ ]:
struct NEMD_longitudinal_forcing{F,T}
   force_profile::F; η::T
end

const ηmin = 0.1
const ηmax = 10.0

function get_forcing()
   η = ηmin + rand()*(ηmax-ηmin) # pick a random η

   println("Creating NEMD forcing with magnitude: $η")
   
   forcing = # fill me in!
   return forcing

end


Now, we must tell Molly how to compute the forces associated with a `NEMD_longitudinal_forcing`. To do so, we need to provide a method for `AtomsCalculators.forces` compatible with our custom interaction type.

- Look at the [Molly documentation](https://juliamolsim.github.io/Molly.jl/stable/documentation/#General-interactions) to understand how to implement custom interactions.
- Complete the following method.

<details>

<summary><b>Answer</b></summary>

```julia
function AtomsCalculators.forces(s::System,inter::NEMD_longitudinal_forcing, neighbors=nothing,step_n=0, n_threads=Threads.nthreads();kwargs...)
    forces_out = ustrip_vec.(similar(s.coords))*force_unit
    @inbounds for i in 1:length(s)
        fx = inter.η * inter.force_profile(s.coords[i][2])
        forces_out[i] = SVector(fx, zero(fx), zero(fx))
    end
    
    return forces_out
end
```
</details>

In [ ]:
function AtomsCalculators.forces(s::System,inter::NEMD_longitudinal_forcing, neighbors=nothing,step_n=0, n_threads=Threads.nthreads();kwargs...)
    forces_out = ustrip_vec.(similar(s.coords))*force_unit
    @inbounds for i in 1:length(s)
        # fill me in!
    end
    
    return forces_out
end

---
### 1.4 - Response flux

We define a function that computes our desired response. In our case, it corresponds to the first empirical Fourier coefficient <a href="#Rk1">(7)</a>, which we recall:

$$
    R(q,p) = \frac{1}{N}\sum_{i=1}^N \frac{p_{i1}}{m}\exp\left(\frac{2i\pi q_{i2}}{L}\right).
$$

Complete the following cell.

<details>
<summary><b>Answer</b></summary>

```julia
function fourier_response(s::System, args...; kwargs...)
    return sum(s.velocities[i][1] * exp(2im * π * s.coords[i][2] / L) for i=1:N)/N
end
```

</details>

In [ ]:
function fourier_response(s::System, args...; kwargs...)
    # fill me in !
end

We also need to define a logger to track the value of `fourier_response` throughout the simulation.

Define a logger named `fourier_logger` using the [`GeneralObservableLogger`](https://juliamolsim.github.io/Molly.jl/stable/api/#Molly.GeneralObservableLogger) constructor. Take care of specifying the return type of `fourier_response`. For simplicity, we will track the value at each time step.

<details>
<summary><b>Answer</b></summary>

```julia
S = typeof((1.0+1.0im)*u"Å/ps")
fourier_logger = GeneralObservableLogger(fourier_response, S, 1)
```

</details>

In [ ]:
fourier_logger = # fill me in !

---
### 1.5 - Simulation setup

We now define the system and choose an integrator.

Checkout the [`System`](https://juliamolsim.github.io/Molly.jl/stable/api/#Molly.System) type in the Molly documentation. Fill in the missing arguments in the constructor.
We use a [`LangevinSplitting`]() integrator, with a timestep $\Delta t=2\,\mathrm{fs}$.

<details>

<summary><b>Answer</b></summary>

```julia
nemd_sys = System(
    atoms=atoms,
    coords=coords,
    boundary=boundary,
    pairwise_inters= (lj_w_cutoff,),
    general_inters= (forcing,),
    loggers= (fourier=fourier_logger,),
    neighbor_finder= nf,
    energy_units= energy_unit,
    force_units= force_unit
)
```

</details>

In [ ]:
forcing = get_forcing()

nemd_sys = # fill me in!

const τ = uconvert(u"ps", σ * sqrt(m / ϵ)); # characteristic timescale
const γ = m/τ # friction parameter
const dt = 2.0u"fs" # time step

langevin_integrator = LangevinSplitting(dt=dt, temperature=T, friction=γ,remove_CM_motion=0,splitting="ABOBA")

Check that `fourier_response(nemd_sys)` returns a complex velocity, with units of `Å/ps`. The functions [`Unitful.unit`](https://painterqubits.github.io/Unitful.jl/stable/manipulations/#Unitful.unit) and [`Unitful.dimension`](https://painterqubits.github.io/Unitful.jl/stable/manipulations/#Unitful.dimension) may be of use here.

---
#### 1.6 - Running the simulation.

Make sure you understand the following function, which performs NEMD runs, using the [`simulate!`](https://juliamolsim.github.io/Molly.jl/stable/api/#Molly.simulate!-Tuple{Any,%20SteepestDescentMinimizer}) function from Molly.

In [ ]:
function run_nemd_sim!(nsteps;progression_freq=1000,thermalization=false,push_results=false)
    
    @showprogress for k=1:(nsteps ÷ progression_freq)
        simulate!(nemd_sys,langevin_integrator,progression_freq,run_loggers = !thermalization)
    end

    vals = values(nemd_sys.loggers.fourier)

    flux,steps = mean(vals),length(vals)

    if push_results && !thermalization
        post_result("NEMD",forcing.η,steps,flux,ρ,T,γ,N,flux/forcing.η)
    end
    
    empty!(nemd_sys.loggers.fourier.history) # clears logger history
    
    return flux
end

Perform `K` independent realizations of the NEMD simulation, consisting of 
- A decorrelation/thermalization run for $t_{\mathrm{eq}}=20\,\mathrm{ps}$ (10000 steps).
- A sampling run for $t_{\mathrm{sim}}=100\,\mathrm{ps}$ (50000 steps).

At the end of each sampling run, collect the return value of `run_nemd_sim!`, which is the trajectory average of the empirical Fourier flux.

Start with `K=1` and `push_results=false`, then, if you are confident your setup is working, adjust `K` as you like and set `push_results=true`. You can interrupt the run if the simulations are taking too long. Do not change the values of $t_{\mathrm{sim}}$ and $t_{\mathrm{eq}}$, as this will affect future error estimates.

<details>
<summary><b>Answer</b></summary>

```julia
K = 20

for k=1:K
    println("Run: $k. Decorrelating")
    run_nemd_sim!(ndecorr;thermalization=true)
    println("Run: $k. logging fourier response.")
    run_nemd_sim!(nsim;push_results=true)
end
```
</details>

In [ ]:
const ndecorr = 10000
const nsim = 50000

K = 1

for k=1:K
    println("Run: $k. Decorrelating")
    # fill me in!
    println("Run: $k. logging fourier response.")
    # fill me in!
end

### 1.7- Plotting the response profile and estimating $\mu$

We will now collect results from everyone,  and construct a NEMD estimator for the shear viscosity, based on a polynomial model of the response profile.
First we fetch the results. As API calls are limited in number, most of them should be used to post simulation data: please do not spam the following cell.

In [ ]:
nemd_results,gk_results = fetch_results(ρ,T,γ,N) # collect nemd results # RUN SPARINGLY !

We now estimate the linear response, using a polynomial model 
$$
\hat R(\eta) = \sum_{k=1}^{d} \hat\theta_k \eta^k,
$$

for the response function $\eta\mapsto\mathfrak{Im}\mathbb{E}_\eta[R]$. We use $\hat\theta_1$ as an estimator of the linear response $\mathfrak{Im}U_1$.
for a $d$ of your choice, fit $\hat R$ to the sampled data, using ordinary least squares. Output the linear response, and plot the results.

<b> Bonus: </b> Add error bars on the $\theta_k$ assuming i.i.d. Gaussian errors on the response (homoscedastic noise model).

<details>
<summary><b> Answer </b></summary>

```julia
results = reduce(hcat,nemd_results)

etas = results[2,:]
Rs = results[3,:]

order = 2 # order of polynomial fit
X = hcat([etas .^k for k=1:order]...)
println(size(X))
Y = Rs

θ = (X'X)\(X'Y)*u"Å/ps" # least squares

# homoscedastic error bars
n = size(X,1)
p = size(X,2)
σ2hat = inv(n-p)*sum(abs2,Y*u"Å/ps"-X*θ)
Σhat = σ2hat*inv(X'X)


inv(X'X)

imU1_nemd = θ[1] ± 1.96√Σhat[1,1]
println("Estimated linear response Im(U1): $(imU1_nemd)")


scatter(xlabel=L"\eta",ylabel="Response",etas,Rs,label="")
plot!(t-> sum(θ[k]*t^k for k=1:order),0,ηmax,label="fit")
```

</details>

In [ ]:
results = reduce(hcat,nemd_results)

etas = results[2,:]
imRs = results[3,:]

order = 2 # order of polynomial fit
X = hcat([etas .^k for k=1:order]...)
println(size(X))
Y = Rs


imU1_nemd = # fill me in!
println("Estimated linear response Im(U1): $(imU1_nemd)")
# plot results

Estimate the shear viscosity, using the formula:

$$
\mu = \overline{\rho}\left(\frac{F_1}{U_1}-\gamma\right)\left(\frac{L}{2\pi}\right)^2,
$$

where $\overline{\rho}=N/L^3$. Estimate the shear viscosity in $\mu\mathrm{Pa}\cdot \mathrm{s}$, given $F_1=\mathrm{i}/2\,\mathrm{pN}$.

<details>
<summary><b>Answer</b></summary>

```julia
imF1 = 0.5u"pN"
μ_nemd = uconvert(u"μPa*s",(N/L^3)*(imF1/imU1_nemd-γ)*(L/2π)^2)
```

</details>

In [ ]:
imF1 = 0.5u"pN"
μ_nemd = # fill me in!

## Method 2: Green-Kubo formula

We now use the Green-Kubo formula, which expresses the Fourier linear response as an equilibrium **dynamical** average
$$
    U_1 = \int_0^{+\infty} \mathbb{E}_0\left[R(q_t,p_t)S(q_0,p_0)\right] dt,
$$

where $R$ is the empirical Fourier flux, and $S$ is the conjugate response

$$
S(q,p) = \frac{\beta}{m} F(q)^\top p.
$$

### 2.1-Defining the conjugate response

We already defined the response $R$ in `fourier_reponse`. Now, define the conjugate response `shear_conj_response`.

<details>

<summary><b>Answer</b></summary>

```julia
const β = inv(kB*T)

function shear_conj_response(s::System, args...; kwargs...)
   return β * sum(s.velocities[i][1] * sin(2π * s.coords[i][2] / L) for i = 1:N) * s.force_units # fill me in!
end
```

</details>

In [ ]:
const β = inv(kB*T)

function shear_conj_response(s::System, args...; kwargs...)
   return # fill me in!
end

### 2.2-Constructing the autocorrelation estimator

The autocorrelation function 
$$
C(t) = \mathbb E_0\left[R(q_t,p_t)S(q_0,p_0)\right]
$$
can be estimated along a single long equilibrium trajectory, using the following estimator
$$
\hat C(k\Delta t) = \frac{1}{N_{\mathrm{sim}}-k+1}\sum_{j=0}^{N_{\mathrm{sim}}-k} R(q^{(j+k)},p^{(j+k)})S(q^{(j)},p^{(j)}),
$$

where $(q^{(i)},p^{(i)})\approx (q_{i\Delta t},p_{i\Delta t})$ for $0\leq i\leq N_{\mathrm{sim}}$ is the numerical trajectory. Convince yourself this is a consistent estimator, assuming that the trajectory is stationary.

We will record the value of this estimator, for $0\leq k\leq N_{\mathrm{corr}}$, where $N_{\mathrm{corr}}$ is a fixed correlation length. One typically requires
$$
1\ll N_{\mathrm{corr}}\ll N_{\mathrm{sim}}.
$$
The first requirement allows to capture the full autocorrelation decay profile, while the second ensures that many samples are recorded for each point on the autocorrelation profile. Therefore, a long simulation needs to be performed.

Molly has the built-in support for these kinds of estimators. Read the documentation for the [`TimeCorrelationLogger`](https://juliamolsim.github.io/Molly.jl/stable/api/#Molly.TimeCorrelationLogger) type. Construct the autocorrelation logger, using `ncorr_gk=7500` for $N_{\mathrm{corr}}$.

<b>Warning : </b> The complex observable should be `observableB` in `TimeCorrelationLogger`, otherwise it will get conjugated. Pay attention to the return types.

<details>
<summary><b>Answer</b></summary>

```julia
ncorr_gk = 7500

typeA = typeof(β*force_unit*u"Å/ps") # dimensions of inverse time
typeB = typeof((1.0 + 1.0im) * u"Å/ps")

gk_correlation_logger = TimeCorrelationLogger(shear_conj_response,fourier_response, typeA,typeB, 1, ncorr_gk)
```
</details>

In [ ]:
const ncorr_gk = 5000

gk_correlation_logger = # fill me in !

### 2.3-System setup

Set up an **equilibrium** system (no forcing).


<details>

<summary> <b>Answer</b></summary>

```julia
gk_sys = System(
    atoms=atoms,
    coords=coords,
    boundary=boundary,
    pairwise_inters=(lj_w_cutoff,),
    loggers=(gk_correlation=gk_correlation_logger,),
    neighbor_finder=nf,
    energy_units=energy_unit,
    force_units=force_unit
)
```
</details>

In [ ]:
gk_sys = # fill me in!

### 2.4-Linear response estimator

We obtain an estimate of the linear response $U_1$ by quadrature rules. A simple and [accurate](https://arxiv.org/abs/1308.5814) (see Figure 1) choice is the trapezoidal rule:

$$
\hat U_1 = \frac{\Delta t}{2}\sum_{k=0}^{N_{\mathrm{corr}}-1} \left[\hat C(k\Delta t)+\hat C((k+1)\Delta t)\right] \approx \int_0^{N_{\mathrm{corr}}\Delta t}C(s)\,\mathrm{d}s.
$$

The **unnormalized** values we want can be obtained with the method `values(::TimeCorrelationLogger;normalize::Bool)`. Complete the function `get_u1_estimate` to compute $\hat U_1$. Give the result in `Å/ps`.

<details>

<summary><b>Answer</b></summary>

```julia
function get_u1_estimate()
    corrs = values(gk_sys.loggers.gk_correlation;normalize=false)
    n = size(corrs,1)
    return uconvert(u"Å/ps",dt*(sum(corrs[2:n-1]) + (first(corrs)+last(corrs))/2))
end
```
</details>

In [ ]:
function get_u1_estimate()
    corrs = values(gk_sys.loggers.gk_correlation;normalize=false)
    return # fill me in!
end

### 2.5-Simulation run

Make sure you understand the method `run_gk_sim!` below.

In [ ]:


"""
For convenience -- resets the correlation logger. Other option would be to redeclare the system.
"""
function clean_gk_logger!()
    zero!(x::AbstractArray) = begin x .= zero(eltype(x)) end

    empty!(gk_sys.loggers.gk_correlation.history_A)
    empty!(gk_sys.loggers.gk_correlation.history_B)
    zero!(gk_sys.loggers.gk_correlation.sum_offset_products)
    gk_sys.loggers.gk_correlation.n_timesteps = 0
    gk_sys.loggers.gk_correlation.sum_A = zero(gk_sys.loggers.gk_correlation.sum_A)
    gk_sys.loggers.gk_correlation.sum_B = zero(gk_sys.loggers.gk_correlation.sum_B)
    gk_sys.loggers.gk_correlation.sum_sq_A = zero(gk_sys.loggers.gk_correlation.sum_sq_A)
    gk_sys.loggers.gk_correlation.sum_sq_B = zero(gk_sys.loggers.gk_correlation.sum_sq_B)
end

function run_gk_sim!(nsteps;thermalization=false,push_results=false)
    simulate!(gk_sys,langevin_integrator,nsteps,run_loggers = !thermalization)
    U1 = get_u1_estimate()

    if push_results && !thermalization
        post_result("GK",0.0,nsteps,0.0u"Å/ps",ρ,T,γ,N,U1)
    end

    vals = values(gk_sys.loggers.gk_correlation;normalize=false)
    clean_gk_logger!()
    
    return vals
end

Run `K` Green-Kubo simulations using the loop below. Start with `K=1` and `push_results=false` in the production run, check `vals`, and if you are confident everything is working, set `K` to a higher value, and `push_results=true`.

In [ ]:
K = 1

const nsteps_eq_gk = 10000
const nsteps_sim_gk = 30*ncorr_gk
typeC = typeof((1.0+1.0im)u"Å/ps^2")

vals = Vector{typeC}[]

@showprogress for k=1:K
    run_gk_sim!(nsteps_eq_gk;thermalization=true)
    push!(vals,run_gk_sim!(nsteps_sim_gk;push_results=false))
end

### 2.6-Plotting correlation profiles

In the cell below, we plot the results from your local simulation runs (accumulated in `vals`). Dark lines correspond to individual realizations, and the red line is the mean. The left plot is the correlation $C(t)$, while the right plot is its integral. Notice the variance of the estimator increases with the integration time.

In [ ]:
n = size(first(vals),1)
k = size(vals,1)

ts = dt*(1:n)

pl_ac = plot(xlabel=L"t",ylabel=L"\mathfrak{Im}\,C(t)",margin=10Plots.mm)
pl_iac = plot(xlabel=L"t",ylabel=L"\mathfrak{Im}\,\int_0^t C(s)\mathrm{d}s",margin=10Plots.mm)

for j=1:k
    plot!(pl_ac,ts,imag.(vals[j]),color=:black,alpha=0.2,label="")
    plot!(pl_iac,ts,dt*imag.(cumsum(vals[j])) |> (x->uconvert.(u"Å/ps",x)) ,color=:black,alpha=0.2,label="")
end

plot!(pl_ac,ts,imag.(mean(vals)),color=:red,label="")
plot!(pl_iac,ts,dt*imag.(cumsum(mean(vals))) |> (x->uconvert.(u"Å/ps",x)),color=:red,label="")

plot(pl_ac,pl_iac,size=(1200,500))

### 2.7-Shear-viscosity estimate

Collect simulation results using the first cell in question 1.7. Estimate the shear viscosity with the Green-Kubo method

In [ ]:
gk_mat_results = reduce(hcat,gk_results)
imU1_gk = (mean(gk_mat_results[2,:]) ± 1.96std(gk_mat_results[2,:]))*u"Å/ps"

imF1 = 0.5u"pN"
μ_gk = # fill me in !

## 3-Comparison with NIST data

The NIST database gives reference experimental values. Compare our results with these measurements [here](https://webbook.nist.gov/cgi/fluid.cgi?D=1410&TLow=80&THigh=90&TInc=1&Digits=5&ID=C7440371&Action=Load&Type=IsoChor&TUnit=K&PUnit=MPa&DUnit=kg%2Fm3&HUnit=kJ%2Fmol&WUnit=m%2Fs&VisUnit=uPa*s&STUnit=N%2Fm&RefState=DEF). Note despite our system being very small ($N=216$), the results are already quite reasonable. More accurate measurements can be performed with larger systems.

In [ ]:
println("====Estimated shear viscosities====")
println("NEMD: ",μ_nemd)
println("Green-Kubo: ",μ_gk)
println("NIST: ", ) # fill me in

### Thanks !

I will keep the API open until available calls run out, feel free to play around with it !